In [ ]:
import numpy as np

In [ ]:
N = 10**2
d = 10
inner_rank = N//2

X = np.dot(np.random.randn(N, inner_rank), np.random.randn(inner_rank, d))
MSE = (X ** 2).mean()
print(f"Mean squared element: {MSE}")

In [ ]:
X_incomplete = X.copy()
# missing entries indicated with NaN
for i in range(N):
    X_incomplete[i, np.random.randint(d)] = np.nan
    

# X is the complete data matrix
# X_incomplete has the same values as X except a subset have been replace with NaN

## matrix completion using fancyimpute

In [ ]:
# idea is to construct lines, find k-centers, find pts closest to 
# those k center for each line, find difference from original X

In [ ]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute

X_knn = KNN(k=d).fit_transform(X_incomplete)

# Slow! Do not execute when n>300
X_filled_nnm = NuclearNormMinimization().fit_transform(X_incomplete)

X_softimpute = SoftImpute(max_iters=10*5, verbose=False).fit_transform(X_incomplete)

# X_iterativeSVD = IterativeSVD(rank=d-1, max_iters=10*5, verbose=False).fit_transform(X_incomplete)

# print mean squared error for the  imputation methods above

nnm_mse = ((X_filled_nnm - X) ** 2).mean()
print("Nuclear norm minimization MSE: %f" % nnm_mse)

softImpute_mse = ((X_softimpute - X) ** 2).mean()
print("SoftImpute MSE: %f" % softImpute_mse)

# iterativeSVD_mse = ((X_iterativeSVD - X) ** 2).mean()
# print("IterativeSVD MSE: %f" % iterativeSVD_mse)

knn_mse = ((X_knn - X) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)

## setup coreset streamer

In [ ]:
import sys
sys.path.insert(1, "./KLines")

import numpy as np
import copy
import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from klines import SetOfLines, SetOfPoints, CorsetForKMeansForLines, CoresetStreamer

# assert(np.version.full_version == '1.16.5')  # later revisions hv slower array lookups

displacements = np.nan_to_num(X_incomplete)

spans = np.nan_to_num(X_incomplete)
spans[spans==0] = 1
spans[spans!=1] = 0

L = SetOfLines(spans, displacements, np.ones(N), np.ones(N))

class ParameterConfig:
    def __init__(self):
        pass
    
config = ParameterConfig()

In [ ]:
## data
k = 2*d
m = 100  # coreset size ~ reduction ratio
tau = 1e-2

config.a_b_approx_minimum_number_of_lines = 100 # constant 100, line 2, algo 2 BI-CRITERIA

config.sample_size_for_a_b_approx = int(m*1.05) # |S| >= m, line 3 of algo 2
                                                # note: there'll be a O(|S|^2) cost while computing algo 1
    
config.farthest_to_centers_rate_in_a_b_approx = 4/11  # opp of 7/11, line 6, algo 2 BI-CRITERIA
config.number_of_remains_multiply_factor = int(math.log(N))//k # this is `b` in algo 2, other paper, set as random here -  how to calculate it?
config.closest_to_median_rate = (1-tau)/(2*k)  # refer line 4, algo 1, other paper

config.median_sample_size = int(N*0.05)    # size of q_i, line 3, algo 2, other paper
config.max_sensitivity_multiply_factor = 100  # for outliers in coresets

config.number_of_remains = 20

SAMPLE_SIZE = 50   

In [ ]:
streamer = CoresetStreamer(SAMPLE_SIZE, k, config)
coreset = streamer.stream(L)
L1 = coreset[0]

_, B, _ = CorsetForKMeansForLines(config).coreset(L1, k, int(L1.get_size()*0.6), True)

X_klines = L.get_projected_centers(B)
klines_mse = ((X - X_klines)**2).mean()

print(f"Klines MSE: {klines_mse}")

In [ ]:
for i in [knn_mse, nnm_mse, softImpute_mse, klines_mse]:
    print(f"{i/MSE}")